In [30]:
# Importación de librerías necesarias
import pandas as pd
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

A continuación, se leen los datos de entrenamiento y prueba proporcionados en dos archivos separados.

In [31]:
# Lectura de los datos de entrenamiento y prueba
train = pd.read_csv('train_dataset.csv')
test = pd.read_csv('test_dataset.csv')

# Separación de las variables dependientes e independientes
X_train = train.drop(columns=['type'], axis=1)
y_train = train['type']

X_test = test.drop(columns=['type'], axis=1)
y_test = test['type']

Observando el dataset de entrenamiento, se identifican un total de 22 variables con las que se espera entrenar el modelo de clasificación, adicionales a la variable `type`, que es la columna donde se encuentran las dos posibles clases en que se pueden identificar los hongos.

In [32]:
# Visualización del dataset de entrenamiento
X_train.head()

,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,x,y,n,t,p,f,c,n,p,e,...,s,w,w,p,w,o,p,k,s,u
1,f,y,g,t,n,f,c,b,p,t,...,s,g,g,p,w,o,p,k,y,d
2,x,y,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,m
3,x,s,w,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,k,a,g
4,x,f,n,t,n,f,c,b,p,t,...,s,p,w,p,w,o,p,n,v,d


In [33]:
# Visualización de las clases del dataset de entrenamiento
y_train.value_counts()

type
e    3167
p    2926
Name: count, dtype: int64

El conjunto de datos tiene un total de 6093 registros, correspondientes a un grupo de hongos de las familias Agaricus y Lepiota, los cuales fueron previamente identificados como comestibles o venenosos (considerando que, para este caso, los hongos de comestibilidad desconocida deben ser clasificados como venenosos).

In [34]:
# Dimensiones del dataset de entrenamiento
train.shape

(6093, 23)

Como se puede observar a continuación, todas las características presentes en el dataset son categóricas, por lo que deben ser convertidas a numéricas con algún tipo de codificación antes de entrenar el modelo.

In [35]:
# Tipo de datos del dataset de entrenamiento
X_train.dtypes

cap_shape                   object
cap_surface                 object
cap_color                   object
bruises                     object
odor                        object
gill_attachment             object
gill_spacing                object
gill_size                   object
gill_color                  object
stalk_shape                 object
stalk_root                  object
stalk_surface_above_ring    object
stalk_surface_below_ring    object
stalk_color_above_ring      object
stalk_color_below_ring      object
veil_type                   object
veil_color                  object
ring_number                 object
ring_type                   object
spore_print_color           object
population                  object
habitat                     object
dtype: object

Para cumplir este propósito será necesario crear un pipeline con dos pasos: transformación de las variables y clasificación. De esta manera se podrá entrenar un modelo sobre el dataset de entrenamiento y predecir sobre el dataset de prueba sin ningún inconveniente.

In [36]:
# Codificación de las variables categóricas
transformer = ColumnTransformer(
    transformers=[
        ('label_encoder', OneHotEncoder(dtype='int'), X_train.columns)
    ]
)

Ahora bien, para el ejercicio de clasificación se empleará un modelo de K-Nearest Neighbors con un parámetro K=3, que se evaluará posteriormente en su resultado.

In [37]:
# Creación del pipeline con el clasificador KNN
pipeline = Pipeline(
    steps=[
        ('transformer', transformer),
        ('classifier', KNeighborsClassifier(n_neighbors=3))
    ]
)

In [44]:
# Métrica para los datos de entrenamiento
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred)

print(f'Accuracy: {accuracy_train}')

Accuracy: 1.0


In [45]:
y_pred = pipeline.predict(X_test)

In [46]:
# Métrica para los datos de prueba
accuracy_test= accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy_test}')

Accuracy: 1.0


In [47]:
# Guardar el modelo
with open('model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)